# a continue to main note book

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 21.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3096341 sha256=0e2e860dac3ee5c977d6fe53909d46dff88a08e2d94662bff7ad8cd9654afe34
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


from collections import defaultdict #data colector

#Surprise: https://surprise.readthedocs.io/en/stable/
import surprise

from surprise.reader import Reader
from surprise import Dataset
from surprise.model_selection import GridSearchCV

  ##CrossValidation
from surprise.model_selection import cross_validate


  ##Matrix Factorization Algorithms
from surprise import SVD
from surprise import NMF

np.random.seed(42) # replicating results

In [3]:
import numpy as np #to help us use numerical functions
import pandas as pd #to help us use functions for dealing with dataframe
import os #provides functions for creating and removing a directory

In order to avoid uploading manually by running below codes we load data from kaggle by using API command

In [4]:
# install kaggle library
! pip install kaggle

In [5]:
#creating a directory named kaggle
! mkdir ~/.kaggle

In [6]:
# copying kaggle.json to new directory
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
# downloading dataset
!kaggle datasets download -d rounakbanik/the-movies-dataset

100% 227M/228M [00:01<00:00, 163MB/s]
100% 228M/228M [00:01<00:00, 140MB/s]


In [9]:
#creating a new directory in order to unzip dataset overthere
!mkdir the-movies-dataset

In [10]:
!unzip 'the-movies-dataset.zip' -d /content/the-movies-dataset

Archive:  the-movies-dataset.zip
  inflating: /content/the-movies-dataset/credits.csv  
  inflating: /content/the-movies-dataset/keywords.csv  
  inflating: /content/the-movies-dataset/links.csv  
  inflating: /content/the-movies-dataset/links_small.csv  
  inflating: /content/the-movies-dataset/movies_metadata.csv  
  inflating: /content/the-movies-dataset/ratings.csv  
  inflating: /content/the-movies-dataset/ratings_small.csv  


In [11]:
from surprise import Dataset, Reader

from surprise.prediction_algorithms.matrix_factorization import SVD

from surprise import accuracy

In [12]:
movies_filename = pd.read_csv('/content/the-movies-dataset/movies_metadata.csv', low_memory = False)
ratings_filename = pd.read_csv('/content/the-movies-dataset/ratings.csv', low_memory = False)

In [13]:
ratings = ratings_filename
movie_md = movies_filename

In [14]:
# movie dataframe with votes more than 100
movie_md = movie_md[movie_md['vote_count']>100]

# removing user with below 10 votes
ratings = ratings.groupby("userId").filter(lambda x: x['userId'].count() >= 10)

# IDs of movies with count more than 100
movie_ids = [int(x) for x in movie_md['id'].values]

# Select ratings of movies with more than 100 counts
ratings = ratings[ratings['movieId'].isin(movie_ids)]

#holding only 1 millions of ratings
ratings = ratings[:1000000]

# Reset Index
ratings.reset_index(inplace=True, drop=True)

# checking new ratings data shape
ratings.shape

(1000000, 4)

In [15]:
movie_md.shape


(6020, 24)

Don't need to do anything else we already did all necessary<br>
but there is point and that's svd based recommender only used when we have a list of movies

In [16]:
ratings.head(1)

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529


In [17]:
def recommender_svd(watch_list, number_of_recommendation):
  df1 = ratings
  for i in range(len(watch_list)):
    df1 = df1.append({'userId' : int(ratings.loc[999999,'userId'])+1, 'movieId' : int(movie_md.loc[movie_md['title'] == watch_list[i], 'id']), 'rating' : 5, 'timestamp' : 0},
                ignore_index = True)

  # Initialize a surprise reader object
  reader = Reader(line_format='user item rating', sep=',', rating_scale=(0,5), skip_lines=1)
  # Load the data
  data = Dataset.load_from_df(ratings[['userId','movieId','rating']], reader=reader)
  # Build trainset object(perform this only when you are using whole dataset to train)
  trainset = data.build_full_trainset()
  # Initialize model
  svd = SVD()
  # cross-validate
  svd.fit(trainset)

  recommendations = []
  user_movie_interactions_matrix = df1.pivot(index='userId', columns='movieId', values='rating')

  non_interacted_movies = user_movie_interactions_matrix.loc[int(ratings.loc[999999,'userId'])+1][user_movie_interactions_matrix.loc[int(ratings.loc[999999,'userId'])+1].isnull()].index.tolist()

  for item_id in non_interacted_movies:
    est = svd.predict(int(ratings.loc[999999,'userId'])+1, item_id).est
    movie_name = movie_md[movie_md['id']==str(item_id)]['title'].values[0]
    recommendations.append((movie_name, est))

  recommendations.sort(key=lambda x: x[1], reverse=True)
  return recommendations[:number_of_recommendation]

In [18]:
watch = ['Toy Story', 'The Dark Knight', 'The Usual Suspects', 'Taxi Driver']
recommender_svd(watch,10)

<ipython-input-17-2447881fefac>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'userId' : int(ratings.loc[999999,'userId'])+1, 'movieId' : int(movie_md.loc[movie_md['title'] == watch_list[i], 'id']), 'rating' : 5, 'timestamp' : 0},
<ipython-input-17-2447881fefac>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'userId' : int(ratings.loc[999999,'userId'])+1, 'movieId' : int(movie_md.loc[movie_md['title'] == watch_list[i], 'id']), 'rating' : 5, 'timestamp' : 0},
<ipython-input-17-2447881fefac>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df1 = df1.append({'userId' : int(ratings.loc[999999,'userId'])+1, 'movieId' : int(movie_md.loc[movie_md['title'] == watch_list[i], 'id']), 'rating' 

[('Stalag 17', 4.345998671928623),
 ('Deep Impact', 4.261080692951831),
 ('Sleepless in Seattle', 4.258549707127334),
 ("Everybody's Fine", 4.24765044176829),
 ('Once Were Warriors', 4.219625339722452),
 ('License to Wed', 4.217947931342291),
 ("On Her Majesty's Secret Service", 4.190029329157614),
 ('Solaris', 4.179323738921387),
 ('Saw III', 4.156069738548317),
 ('Laura', 4.1528795719693905)]